### Install detectron2

In [1]:
!pip install pyyaml
!pip3 install ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 6.6 MB/s eta 0:00:00


In [2]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git@ffff8ac'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git (to revision ffff8ac) to /tmp/pip-req-build-got6lg4a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-got6lg4a
  Running command git checkout -q ffff8ac
  Resolved https://github.com/facebookresearch/detectron2.git to commit ffff8ac
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.1.1-cp310-cp310-linux_x86_64.whl size=7416562 sha256=e97300913768b9afe6fd52f1bca8d540a7ccde1b0bb9d543353f2c6edc4fb199
  Stored in directory: /tmp/pip-ephem-wheel-cache-onn42m51/wheels/10/70/32/3

In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [4]:
# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

In [5]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

### Getting data

In [8]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [11]:
#!cp -r /content/gdrive/MyDrive/OpenViIC-dataset/images.zip .
#!cp -r /content/gdrive/MyDrive/OpenViIC/OpenViIC/images/train.zip .
!cp -r /content/gdrive/MyDrive/DS204/Final/dataset/OpenViIC/images/train.zip .

In [12]:
!unzip images.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/00000005952.jpg   
  inflating: train/00000003829.png   
  inflating: train/00000007866.jpg   
  inflating: train/00000010215.jpg   
  inflating: train/00000004957.jpg   
  inflating: train/00000002660.jpg   
  inflating: train/00000004850.jpg   
  inflating: train/00000004897.jpg   
  inflating: train/00000000267.jpg   
  inflating: train/00000005338.jpg   
  inflating: train/00000012827.jpg   
  inflating: train/00000010417.jpg   
  inflating: train/00000010298.jpg   
  inflating: train/00000002033.jpg   
  inflating: train/00000005645.jpg   
  inflating: train/00000011693.png   
  inflating: train/00000004326.jpg   
  inflating: train/00000012214.jpg   
  inflating: train/00000001824.jpg   
  inflating: train/00000004944.jpg   
  inflating: train/00000006410.jpg   
  inflating: train/00000008275.jpg   
  inflating: train/00000000532.jpg   
  inflating: train/00000000367.jpg   
  inflating: train/00000006565.jpg   

### Extract features

In [13]:
!git clone https://github.com/facebookresearch/grid-feats-vqa.git

Cloning into 'grid-feats-vqa'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 76 (delta 25), reused 22 (delta 22), pack-reused 42
Unpacking objects: 100% (76/76), 29.79 KiB | 1.42 MiB/s, done.


In [14]:
cd grid-feats-vqa

/content/grid-feats-vqa


Modified class AttributeStandardROIHeads in grid_feats/roi_heads.py as follow:
```python
...
from detectron2.structures import Boxes, ImageList, Instances
from typing import Optional, Dict, Tuple, List
...

class AttributeStandardROIHeads(AttributeROIHeads, StandardROIHeads):
    """
    An extension of StandardROIHeads to include attribute prediction.
    """
    def __init__(self, cfg, input_shape):
        super(StandardROIHeads, self).__init__(cfg, input_shape)
        self._init_box_head(cfg, input_shape)
        self._init_mask_head(cfg, input_shape)
        self._init_keypoint_head(cfg, input_shape)

    def _init_box_head(self, cfg, input_shape):
        # fmt: off
        pooler_resolution        = cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION
        pooler_scales            = tuple(1.0 / input_shape[k].stride for k in self.in_features)
        sampling_ratio           = cfg.MODEL.ROI_BOX_HEAD.POOLER_SAMPLING_RATIO
        pooler_type              = cfg.MODEL.ROI_BOX_HEAD.POOLER_TYPE
        self.train_on_pred_boxes = cfg.MODEL.ROI_BOX_HEAD.TRAIN_ON_PRED_BOXES
        self.attribute_on        = cfg.MODEL.ATTRIBUTE_ON
        # fmt: on

        in_channels = [input_shape[f].channels for f in self.in_features]
        assert len(set(in_channels)) == 1, in_channels
        in_channels = in_channels[0]

        self.box_pooler = ROIPooler(
            output_size=pooler_resolution,
            scales=pooler_scales,
            sampling_ratio=sampling_ratio,
            pooler_type=pooler_type,
        )
        self.box_head = build_box_head(
            cfg, ShapeSpec(channels=in_channels, height=pooler_resolution, width=pooler_resolution)
        )
        self.box_predictor = FastRCNNOutputLayers(cfg, self.box_head.output_shape)

        if self.attribute_on:
            self.attribute_predictor = AttributePredictor(cfg, self.box_head.output_shape.channels)

    def forward(
        self,
        images: ImageList,
        features: Dict[str, torch.Tensor],
        proposals: List[Instances],
        targets: Optional[List[Instances]] = None,
    ) -> Tuple[List[Instances], Dict[str, torch.Tensor]]:
        """
        See :class:`ROIHeads.forward`.
        """
        del images
        if self.training:
            assert targets
            proposals = self.label_and_sample_proposals(proposals, targets)
        del targets

        if self.training:
            losses = self._forward_box(features, proposals)
            # Usually the original proposals used by the box head are used by the mask, keypoint
            # heads. But when `self.train_on_pred_boxes is True`, proposals will contain boxes
            # predicted by the box head.
            losses.update(self._forward_mask(features, proposals))
            losses.update(self._forward_keypoint(features, proposals))
            return proposals, losses
        else:
            pred_instances, box_features = self._forward_box(features, proposals)
            # During inference cascaded prediction is used: the mask and keypoints heads are only
            # applied to the top scoring box detections.
            # pred_instances = self.forward_with_given_boxes(features, pred_instances)
            return pred_instances, box_features

    def _forward_box(self, features, proposals):
        features = [features[f] for f in self.in_features]
        box_features = self.box_pooler(features, [x.proposal_boxes for x in proposals])
        box_features = self.box_head(box_features)
        predictions = self.box_predictor(box_features)

        if self.training:
            if self.train_on_pred_boxes:
                with torch.no_grad():
                    pred_boxes = self.box_predictor.predict_boxes_for_gt_classes(
                        predictions, proposals
                    )
                    for proposals_per_image, pred_boxes_per_image in zip(proposals, pred_boxes):
                        proposals_per_image.proposal_boxes = Boxes(pred_boxes_per_image)
            losses = self.box_predictor.losses(predictions, proposals)
            if self.attribute_on:
                losses.update(self.forward_attribute_loss(proposals, box_features))
                del box_features

            return losses
        else:
            pred_instances, kept_indices = self.box_predictor.inference(predictions, proposals)
            return pred_instances, box_features[kept_indices[0].tolist()]

    def get_conv5_features(self, features):
        assert len(self.in_features) == 1

        features = [features[f] for f in self.in_features]
        return features[0]
```

Create a file called extract_features.py with the following contents:
```python
import argparse
import json
import os
import torch
from torch import nn
from tqdm import tqdm
import numpy as np
from PIL import Image, ImageFile
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import Dataset, DataLoader
from fvcore.common.file_io import PathManager

from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import default_setup
from detectron2.evaluation import inference_context
from detectron2.modeling import build_model
from detectron2.modeling.meta_arch.rcnn import GeneralizedRCNN
from detectron2.modeling.proposal_generator.rpn import RPN
from grid_feats.roi_heads import AttributeStandardROIHeads

from grid_feats import (
    add_attribute_config,
    build_detection_test_loader_with_attributes,
)

ImageFile.LOAD_TRUNCATED_IMAGES = True

device = "cuda" if torch.cuda.is_available() else "cpu"

class ImageDataset(Dataset):
    def __init__(self, image_dir):
        self.data = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
        self.image_ids = [int(filename.split(".")[0]) for filename in os.listdir(image_dir)]

    def __len__(self):
        return len(self.data)

    def read_image(self, filename):
        image = Image.open(filename).convert("RGB")
        image = np.asarray(image)
        image = torch.as_tensor(np.ascontiguousarray(image.transpose(2, 0, 1)), dtype=torch.float32)

        return image

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        filename = self.data[idx]
        image = self.read_image(filename)
        _, h, w = image.shape

        return {
            "id": image_id,
            "image": image,
            "width": w,
            "height": h
        }

class CustomizedRCNN(GeneralizedRCNN):
    def __init__(self, cfg):
        super(CustomizedRCNN, self).__init__(cfg)

        self.adaptive = nn.AdaptiveAvgPool2d((7, 7))

    def forward(self, batched_inputs):
        """
        Args:
            batched_inputs: a list, batched outputs of :class:`DatasetMapper` .
                Each item in the list contains the inputs for one image.
                For now, each item in the list is a dict that contains:

                * image: Tensor, image in (C, H, W) format.
                * instances (optional): groundtruth :class:`Instances`
                * proposals (optional): :class:`Instances`, precomputed proposals.

                Other information that's included in the original dicts, such as:

                * "height", "width" (int): the output resolution of the model, used in inference.
                  See :meth:`postprocess` for details.
        """
        images = self.preprocess_image(batched_inputs)
        grid_features = self.backbone(images.tensor)

        proposals, _ = self.proposal_generator(images, grid_features, None)

        results, box_features = self.roi_heads(images, grid_features, proposals, None)

        grid_features = self.roi_heads.get_conv5_features(grid_features)
        grid_features = self.adaptive(grid_features)

        return results, grid_features, box_features

def custom_collate_fn(samples):
    return samples

def get_relative_pos(x, batch_size, norm_len):
    x = x.view(1, -1, 1).expand(batch_size, -1, -1)
    return x / norm_len

def get_grids_position(batch_size, seq_len, grid_size):
    assert seq_len == grid_size[0] * grid_size[1]

    # record the pos of each grid according to the form of region box
    x = torch.arange(0, grid_size[0]).float().cuda()
    y = torch.arange(0, grid_size[1]).float().cuda()

    px_min = x.view(-1, 1).expand(-1, grid_size[0]).contiguous().view(-1)
    py_min = y.view(1, -1).expand(grid_size[1], -1).contiguous().view(-1)

    px_max = px_min + 1
    py_max = py_min + 1

    # scale pos into the range (0 ~ 1)
    rpx_min = get_relative_pos(px_min, batch_size, grid_size[0])
    rpy_min = get_relative_pos(py_min, batch_size, grid_size[1])

    rpx_max = get_relative_pos(px_max, batch_size, grid_size[0])
    rpy_max = get_relative_pos(py_max, batch_size, grid_size[1])

    boxes = torch.cat([rpx_min, rpy_min, rpx_max, rpy_max], dim=-1)  # (bs, n, 4)

    return boxes

def extract_grid_feature_argument_parser():
    parser = argparse.ArgumentParser(description="Grid feature extraction")
    parser.add_argument("--config-file", default="", metavar="FILE", help="path to config file")
    parser.add_argument(
        "opts",
        help="Modify config options using the command-line",
        default=None,
        nargs=argparse.REMAINDER,
    )
    return parser

def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    add_attribute_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    # force the final residual block to have dilations 1
    cfg.MODEL.RESNETS.RES5_DILATION = 1
    cfg.freeze()
    default_setup(cfg, args)
    return cfg

def main(args):
    cfg = setup(args)

    dataset = ImageDataset("images")
    dataloader = DataLoader(
        dataset,
        shuffle=True,
        batch_size=1,
        num_workers=2,
        collate_fn=custom_collate_fn
    )

    model = CustomizedRCNN(cfg).to(device)
    DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
        cfg.MODEL.WEIGHTS, resume=True
    )
    model.eval()
    item = items[0]
    for items in tqdm(dataloader, desc="Extracting"):
        with torch.no_grad():
            instances, grid_features, region_features = model(items)
            seq_len, region_dim = region_features.shape
            region_boxes = instances[0].get("pred_boxes").tensor
            if seq_len == 0:
              region_features = torch.ones((1, region_dim))
              region_boxes = torch.ones((1, 4)).cuda()
            w = item["width"]
            h = item["height"]
            region_boxes = region_boxes / torch.tensor([w, h, w, h]).cuda()
            bs, grid_dim, grid_size, _ = grid_features.shape
            grid_features = grid_features.reshape(grid_dim, -1).permute(1, 0)
            grid_boxes = get_grids_position(bs, grid_size**2, (grid_size, grid_size)).squeeze(0)
            item = items[0]
            features = {
                "id": item["id"],
                "width": item["width"],
                "height": item["height"],
                "region_features": region_features.detach().cpu().numpy(),
                "region_boxes": region_boxes.detach().cpu().numpy(),
                "grid_features": grid_features.detach().cpu().numpy(),
                "grid_boxes": grid_boxes.detach().cpu().numpy()
            }
            np.save(f"x152++_faster_rcnn/{item['id']}.npy", features, allow_pickle=True)

if __name__ == "__main__":
    args = extract_grid_feature_argument_parser().parse_args()
    main(args)
```

Create a file called extract_features.py with the following contents:
```python
import argparse
import json
import os
import torch
from torch import nn
from tqdm import tqdm
import numpy as np
from PIL import Image, ImageFile
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import Dataset, DataLoader
from fvcore.common.file_io import PathManager

from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import default_setup
from detectron2.evaluation import inference_context
from detectron2.modeling import build_model
from detectron2.modeling.meta_arch.rcnn import GeneralizedRCNN
from detectron2.modeling.proposal_generator.rpn import RPN
from grid_feats.roi_heads import AttributeStandardROIHeads

from grid_feats import (
    add_attribute_config,
    build_detection_test_loader_with_attributes,
)

ImageFile.LOAD_TRUNCATED_IMAGES = True

device = "cuda" if torch.cuda.is_available() else "cpu"

class ImageDataset(Dataset):
    def __init__(self, image_dir):
        self.data = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
        self.image_ids = [int(filename.split(".")[0]) for filename in os.listdir(image_dir)]

    def __len__(self):
        return len(self.data)

    def read_image(self, filename):
        image = Image.open(filename).convert("RGB")
        image = np.asarray(image)
        image = torch.as_tensor(np.ascontiguousarray(image.transpose(2, 0, 1)), dtype=torch.float32)

        return image

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        filename = self.data[idx]
        image = self.read_image(filename)
        _, h, w = image.shape

        return {
            "id": image_id,
            "image": image,
            "width": w,
            "height": h
        }

class CustomizedRCNN(GeneralizedRCNN):
    def __init__(self, cfg):
        super(CustomizedRCNN, self).__init__(cfg)

        self.adaptive = nn.AdaptiveAvgPool2d((7, 7))

    def forward(self, batched_inputs):
        """
        Args:
            batched_inputs: a list, batched outputs of :class:`DatasetMapper` .
                Each item in the list contains the inputs for one image.
                For now, each item in the list is a dict that contains:

                * image: Tensor, image in (C, H, W) format.
                * instances (optional): groundtruth :class:`Instances`
                * proposals (optional): :class:`Instances`, precomputed proposals.

                Other information that's included in the original dicts, such as:

                * "height", "width" (int): the output resolution of the model, used in inference.
                  See :meth:`postprocess` for details.
        """
        images = self.preprocess_image(batched_inputs)
        grid_features = self.backbone(images.tensor)

        proposals, _ = self.proposal_generator(images, grid_features, None)

        results, box_features = self.roi_heads(images, grid_features, proposals, None)

        grid_features = self.roi_heads.get_conv5_features(grid_features)
        grid_features = self.adaptive(grid_features)

        return results, grid_features, box_features

def custom_collate_fn(samples):
    return samples

def get_relative_pos(x, batch_size, norm_len):
    x = x.view(1, -1, 1).expand(batch_size, -1, -1)
    return x / norm_len

def get_grids_position(batch_size, seq_len, grid_size):
    assert seq_len == grid_size[0] * grid_size[1]

    # record the pos of each grid according to the form of region box
    x = torch.arange(0, grid_size[0]).float().cuda()
    y = torch.arange(0, grid_size[1]).float().cuda()

    px_min = x.view(-1, 1).expand(-1, grid_size[0]).contiguous().view(-1)
    py_min = y.view(1, -1).expand(grid_size[1], -1).contiguous().view(-1)

    px_max = px_min + 1
    py_max = py_min + 1

    # scale pos into the range (0 ~ 1)
    rpx_min = get_relative_pos(px_min, batch_size, grid_size[0])
    rpy_min = get_relative_pos(py_min, batch_size, grid_size[1])

    rpx_max = get_relative_pos(px_max, batch_size, grid_size[0])
    rpy_max = get_relative_pos(py_max, batch_size, grid_size[1])

    boxes = torch.cat([rpx_min, rpy_min, rpx_max, rpy_max], dim=-1)  # (bs, n, 4)

    return boxes

def extract_grid_feature_argument_parser():
    parser = argparse.ArgumentParser(description="Grid feature extraction")
    parser.add_argument("--config-file", default="", metavar="FILE", help="path to config file")
    parser.add_argument(
        "opts",
        help="Modify config options using the command-line",
        default=None,
        nargs=argparse.REMAINDER,
    )
    return parser

def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    add_attribute_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    # force the final residual block to have dilations 1
    cfg.MODEL.RESNETS.RES5_DILATION = 1
    cfg.freeze()
    default_setup(cfg, args)
    return cfg

def main(args):
    cfg = setup(args)

    dataset = ImageDataset("images")
    dataloader = DataLoader(
        dataset,
        shuffle=True,
        batch_size=1,
        num_workers=2,
        collate_fn=custom_collate_fn
    )

    model = CustomizedRCNN(cfg).to(device)
    DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
        cfg.MODEL.WEIGHTS, resume=True
    )
    model.eval()
    item = items[0]
    for items in tqdm(dataloader, desc="Extracting"):
        with torch.no_grad():
            instances, grid_features, region_features = model(items)
            seq_len, region_dim = region_features.shape
            region_boxes = instances[0].get("pred_boxes").tensor
            if seq_len == 0:
              region_features = torch.ones((1, region_dim))
              region_boxes = torch.ones((1, 4)).cuda()
            w = item["width"]
            h = item["height"]
            region_boxes = region_boxes / torch.tensor([w, h, w, h]).cuda()
            bs, grid_dim, grid_size, _ = grid_features.shape
            grid_features = grid_features.reshape(grid_dim, -1).permute(1, 0)
            grid_boxes = get_grids_position(bs, grid_size**2, (grid_size, grid_size)).squeeze(0)
            item = items[0]
            features = {
                "id": item["id"],
                "width": item["width"],
                "height": item["height"],
                "region_features": region_features.detach().cpu().numpy(),
                "region_boxes": region_boxes.detach().cpu().numpy(),
                "grid_features": grid_features.detach().cpu().numpy(),
                "grid_boxes": grid_boxes.detach().cpu().numpy()
            }
            np.save(f"x152++_faster_rcnn/{item['id']}.npy", features, allow_pickle=True)

if __name__ == "__main__":
    args = extract_grid_feature_argument_parser().parse_args()
    main(args)
```

Create a file called extract_features.py with the following contents:
```python
import argparse
import json
import os
import torch
from torch import nn
from tqdm import tqdm
import numpy as np
from PIL import Image, ImageFile
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import Dataset, DataLoader
from fvcore.common.file_io import PathManager

from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import default_setup
from detectron2.evaluation import inference_context
from detectron2.modeling import build_model
from detectron2.modeling.meta_arch.rcnn import GeneralizedRCNN
from detectron2.modeling.proposal_generator.rpn import RPN
from grid_feats.roi_heads import AttributeStandardROIHeads

from grid_feats import (
    add_attribute_config,
    build_detection_test_loader_with_attributes,
)

ImageFile.LOAD_TRUNCATED_IMAGES = True

device = "cuda" if torch.cuda.is_available() else "cpu"

class ImageDataset(Dataset):
    def __init__(self, image_dir):
        self.data = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
        self.image_ids = [int(filename.split(".")[0]) for filename in os.listdir(image_dir)]

    def __len__(self):
        return len(self.data)

    def read_image(self, filename):
        image = Image.open(filename).convert("RGB")
        image = np.asarray(image)
        image = torch.as_tensor(np.ascontiguousarray(image.transpose(2, 0, 1)), dtype=torch.float32)

        return image

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        filename = self.data[idx]
        image = self.read_image(filename)
        _, h, w = image.shape

        return {
            "id": image_id,
            "image": image,
            "width": w,
            "height": h
        }

class CustomizedRCNN(GeneralizedRCNN):
    def __init__(self, cfg):
        super(CustomizedRCNN, self).__init__(cfg)

        self.adaptive = nn.AdaptiveAvgPool2d((7, 7))

    def forward(self, batched_inputs):
        """
        Args:
            batched_inputs: a list, batched outputs of :class:`DatasetMapper` .
                Each item in the list contains the inputs for one image.
                For now, each item in the list is a dict that contains:

                * image: Tensor, image in (C, H, W) format.
                * instances (optional): groundtruth :class:`Instances`
                * proposals (optional): :class:`Instances`, precomputed proposals.

                Other information that's included in the original dicts, such as:

                * "height", "width" (int): the output resolution of the model, used in inference.
                  See :meth:`postprocess` for details.
        """
        images = self.preprocess_image(batched_inputs)
        grid_features = self.backbone(images.tensor)

        proposals, _ = self.proposal_generator(images, grid_features, None)

        results, box_features = self.roi_heads(images, grid_features, proposals, None)

        grid_features = self.roi_heads.get_conv5_features(grid_features)
        grid_features = self.adaptive(grid_features)

        return results, grid_features, box_features

def custom_collate_fn(samples):
    return samples

def get_relative_pos(x, batch_size, norm_len):
    x = x.view(1, -1, 1).expand(batch_size, -1, -1)
    return x / norm_len

def get_grids_position(batch_size, seq_len, grid_size):
    assert seq_len == grid_size[0] * grid_size[1]

    # record the pos of each grid according to the form of region box
    x = torch.arange(0, grid_size[0]).float().cuda()
    y = torch.arange(0, grid_size[1]).float().cuda()

    px_min = x.view(-1, 1).expand(-1, grid_size[0]).contiguous().view(-1)
    py_min = y.view(1, -1).expand(grid_size[1], -1).contiguous().view(-1)

    px_max = px_min + 1
    py_max = py_min + 1

    # scale pos into the range (0 ~ 1)
    rpx_min = get_relative_pos(px_min, batch_size, grid_size[0])
    rpy_min = get_relative_pos(py_min, batch_size, grid_size[1])

    rpx_max = get_relative_pos(px_max, batch_size, grid_size[0])
    rpy_max = get_relative_pos(py_max, batch_size, grid_size[1])

    boxes = torch.cat([rpx_min, rpy_min, rpx_max, rpy_max], dim=-1)  # (bs, n, 4)

    return boxes

def extract_grid_feature_argument_parser():
    parser = argparse.ArgumentParser(description="Grid feature extraction")
    parser.add_argument("--config-file", default="", metavar="FILE", help="path to config file")
    parser.add_argument(
        "opts",
        help="Modify config options using the command-line",
        default=None,
        nargs=argparse.REMAINDER,
    )
    return parser

def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    add_attribute_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    # force the final residual block to have dilations 1
    cfg.MODEL.RESNETS.RES5_DILATION = 1
    cfg.freeze()
    default_setup(cfg, args)
    return cfg

def main(args):
    cfg = setup(args)

    dataset = ImageDataset("images")
    dataloader = DataLoader(
        dataset,
        shuffle=True,
        batch_size=1,
        num_workers=2,
        collate_fn=custom_collate_fn
    )

    model = CustomizedRCNN(cfg).to(device)
    DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
        cfg.MODEL.WEIGHTS, resume=True
    )
    model.eval()
    item = items[0]
    for items in tqdm(dataloader, desc="Extracting"):
        with torch.no_grad():
            instances, grid_features, region_features = model(items)
            seq_len, region_dim = region_features.shape
            region_boxes = instances[0].get("pred_boxes").tensor
            if seq_len == 0:
              region_features = torch.ones((1, region_dim))
              region_boxes = torch.ones((1, 4)).cuda()
            w = item["width"]
            h = item["height"]
            region_boxes = region_boxes / torch.tensor([w, h, w, h]).cuda()
            bs, grid_dim, grid_size, _ = grid_features.shape
            grid_features = grid_features.reshape(grid_dim, -1).permute(1, 0)
            grid_boxes = get_grids_position(bs, grid_size**2, (grid_size, grid_size)).squeeze(0)
            item = items[0]
            features = {
                "id": item["id"],
                "width": item["width"],
                "height": item["height"],
                "region_features": region_features.detach().cpu().numpy(),
                "region_boxes": region_boxes.detach().cpu().numpy(),
                "grid_features": grid_features.detach().cpu().numpy(),
                "grid_boxes": grid_boxes.detach().cpu().numpy()
            }
            np.save(f"x152++_faster_rcnn/{item['id']}.npy", features, allow_pickle=True)

if __name__ == "__main__":
    args = extract_grid_feature_argument_parser().parse_args()
    main(args)
```

In [15]:
!wget https://dl.fbaipublicfiles.com/grid-feats-vqa/X-152pp/X-152pp.pth

--2023-06-13 07:34:18--  https://dl.fbaipublicfiles.com/grid-feats-vqa/X-152pp/X-152pp.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.162.103, 52.84.162.20, 52.84.162.119, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.162.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1421069055 (1.3G) [application/octet-stream]
Saving to: ‘X-152pp.pth’

X-152pp.pth         100%[===================>]   1.32G  47.9MB/s    in 25s     

2023-06-13 07:34:44 (54.4 MB/s) - ‘X-152pp.pth’ saved [1421069055/1421069055]



Change the MODEL.WEIGHTS to "X-152pp.pth" in configs/X-152-challenge.yaml

In [16]:
!mkdir x152++_faster_rcnn

In [19]:
!python extract_features.py --config-file configs/X-152-challenge.yaml

[06/13 07:38:16 detectron2]: Rank of current process: 0. World size: 1
[06/13 07:38:17 detectron2]: Environment info:
------------------------  -----------------------------------------------------------------
sys.platform              linux
Python                    3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
numpy                     1.22.4
detectron2                0.1.1 @/usr/local/lib/python3.10/dist-packages/detectron2
detectron2 compiler       GCC 9.4
detectron2 CUDA compiler  11.8
detectron2 arch flags     sm_75
DETECTRON2_ENV_MODULE     <not set>
PyTorch                   2.0.1+cu118 @/usr/local/lib/python3.10/dist-packages/torch
PyTorch debug build       False
CUDA available            True
GPU 0                     Tesla T4
CUDA_HOME                 /usr/local/cuda
NVCC                      Build cuda_11.8.r11.8/compiler.31833905_0
Pillow                    8.4.0
torchvision               0.15.2+cu118 @/usr/local/lib/python3.10/dist-packages/torchvision
torchvision arc

In [20]:
import os
import numpy as np
from tqdm import tqdm

for feature_file in tqdm(os.listdir("x152++_faster_rcnn")):
  features = np.load(os.path.join("x152++_faster_rcnn", feature_file), allow_pickle=True)[()]
  region_features = features["region_features"]
  region_boxes = features["region_boxes"]
  if region_boxes.max() > 1 or region_boxes.min() > 1:
    raise
#   width = features["width"]
#   height = features["height"]
#   region_boxes = region_boxes / np.array([width, height, width, height])
#   features = {
#                 **features,
#                 "region_features": region_features,
#                 "region_boxes": region_boxes
#             }
#   np.save(os.path.join("x152++_faster_rcnn", feature_file), features, allow_pickle=True)

100%|██████████| 9056/9056 [00:44<00:00, 202.49it/s]


In [21]:
!zip x152++_faster_rcnn.zip -r x152++_faster_rcnn

Streaming output truncated to the last 5000 lines.
  adding: x152++_faster_rcnn/9776.npy (deflated 39%)
  adding: x152++_faster_rcnn/3309.npy (deflated 42%)
  adding: x152++_faster_rcnn/3222.npy (deflated 40%)
  adding: x152++_faster_rcnn/5203.npy (deflated 43%)
  adding: x152++_faster_rcnn/6623.npy (deflated 36%)
  adding: x152++_faster_rcnn/4451.npy (deflated 45%)
  adding: x152++_faster_rcnn/11116.npy (deflated 43%)
  adding: x152++_faster_rcnn/9463.npy (deflated 39%)
  adding: x152++_faster_rcnn/1364.npy (deflated 41%)
  adding: x152++_faster_rcnn/6602.npy (deflated 45%)
  adding: x152++_faster_rcnn/6364.npy (deflated 41%)
  adding: x152++_faster_rcnn/7375.npy (deflated 45%)
  adding: x152++_faster_rcnn/664.npy (deflated 43%)
  adding: x152++_faster_rcnn/3474.npy (deflated 28%)
  adding: x152++_faster_rcnn/8858.npy (deflated 45%)
  adding: x152++_faster_rcnn/915.npy (deflated 36%)
  adding: x152++_faster_rcnn/10076.npy (deflated 42%)
  adding: x152++_faster_rcnn/8819.npy (deflated 

In [ ]:
#!mv x152++_faster_rcnn.zip /content/gdrive/MyDrive/OpenViIC/OpenViIC/features/

In [22]:
!mv x152++_faster_rcnn.zip /content/gdrive/MyDrive/DS204/Final/dataset/OpenViIC/features